In [30]:
import random

In [207]:
class ProblemaGenetico(object):
        def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness,longitud_individuos, tiempos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            self.tiempos = tiempos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma, self.tiempos, self.longitud_individuos)
            return valoracion

In [208]:
def fun_dec(cromosoma, n, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i] == 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l

In [209]:
def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad
    Durante la generación de poblaciones, se decide que individuo puede hacer cada tarea, al cruzarse, 
    los nuevos hijos pueden nacer sabiendo hacer tareas que uno de los padres sabe pero otro no"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2]+cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2]+cromosoma1[l2//2:l1]
    return [cruce1,cruce2]


"""  """
def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob
        A la hora de generar una poblacion, se decide que trabajadores pueden realizar cada tarea
        a la hora de mutar, esta asignacion puede cambiar durante la mutacion,
         interpretando así que el trabajador ha aprendido a hacer la nueva tarea """
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        print("HE MUTADO")
        cromosoma[p] =  (cromosoma[p]+1)%l
    return cromosoma

def fun_fitness(cromosoma, tiempos, tam):
    """Función de valoración que calcula cuanto tiempo tardara una determinada configuracion"""
    n = 0
    for i in range(tam[0]):
        n += tiempos[i][cromosoma[i]]
       

    return n

In [210]:
def poblacion_inicial(problema_genetico, size):
    
    gen = [0,1]
    poblacion = [] #La poblacion final que se recibe
    
    tareas = [] #Cada elemento de la lista representa a un trabajador que sera quien realiza la tarea de su posicion en la lista
    numTrab = problema_genetico.longitud_individuos[1]
    numTareas = problema_genetico.longitud_individuos[0]
    
    plantilla = [] #Lista con todas las capacidades
    capacidad = [ [0 for i in range(numTrab)] for j in range(numTareas)]
    i = 0
    j = 0
    
    
    for s in range(size):
        while i < numTareas:
            #se genera la lista con las capacidades de cada uno de los trabajadores para cada tarea
            while j < numTrab:
              capacidad[i][j] = random.choice(gen)
              j += 1
            j = 0
            #se comprueba que cada tarea tenga al menos un trabajador que la realice, si no la tiene se le asigna uno aleagorio
            if 1 not in capacidad[i]:
                capacidad[i][random.choice(range(numTrab))] = 1
            i += 1
        plantilla.append(capacidad)
        
        for k in range(numTareas):
            t = random.choice(range(numTrab))
            
            while(capacidad[k][t] != 1):
                t = random.choice(range(numTrab))
            tareas.append(t)

        poblacion.append(tareas)
        tareas = []
    return poblacion
            




In [211]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados 

In [234]:
tam = [30, 6] #30 tareas para 6 trabajadores
t1 = [ [(random.choice(range(1, 20))) for i in range(tam[1])] for j in range(tam[0])] #se le da un tiempo aleatorio a cada tarea dependiendo de                                                                                         que trabajador la realiza
p = ProblemaGenetico([0, 1], fun_dec, fun_mutar, fun_cruzar, fun_fitness, tam, t1)
size = 50
poblacion = poblacion_inicial(p, size)
s = seleccion_por_torneo(p, poblacion, 1, size, min)
time = 0
for i in range(tam[0]):
    print("Tarea", i ,"empezada en el instante", time, "por el empleado", s[0][i])
    time += t1[i][s[0][i]]
print("Tiempo total invertido:", p.fitness(s[0]))

Tarea 0 empezada en el instante 0 por el empleado 1
Tarea 1 empezada en el instante 4 por el empleado 0
Tarea 2 empezada en el instante 9 por el empleado 1
Tarea 3 empezada en el instante 13 por el empleado 0
Tarea 4 empezada en el instante 19 por el empleado 1
Tarea 5 empezada en el instante 38 por el empleado 1
Tarea 6 empezada en el instante 53 por el empleado 2
Tarea 7 empezada en el instante 72 por el empleado 1
Tarea 8 empezada en el instante 75 por el empleado 0
Tarea 9 empezada en el instante 81 por el empleado 4
Tarea 10 empezada en el instante 83 por el empleado 0
Tarea 11 empezada en el instante 85 por el empleado 0
Tarea 12 empezada en el instante 90 por el empleado 5
Tarea 13 empezada en el instante 101 por el empleado 3
Tarea 14 empezada en el instante 116 por el empleado 4
Tarea 15 empezada en el instante 119 por el empleado 2
Tarea 16 empezada en el instante 124 por el empleado 5
Tarea 17 empezada en el instante 126 por el empleado 4
Tarea 18 empezada en el instante 133

In [231]:
tam = [4, 4]
time = [[2, 3, 6, 8],
           [1, 3, 9, 2],
           [4, 8, 1, 5],
           [2, 1, 3, 3]]
test = ProblemaGenetico(range(tam[1]), fun_dec, fun_mutar, fun_cruzar, fun_fitness, tam, t1)

Tarea 0 empezada en el instante 0 por el empleado 1
Tarea 1 empezada en el instante 2 por el empleado 1
Tarea 2 empezada en el instante 12 por el empleado 4
Tarea 3 empezada en el instante 15 por el empleado 2
Tarea 4 empezada en el instante 32 por el empleado 1
Tarea 5 empezada en el instante 41 por el empleado 3
Tarea 6 empezada en el instante 55 por el empleado 3
Tarea 7 empezada en el instante 57 por el empleado 4
Tarea 8 empezada en el instante 58 por el empleado 0
Tarea 9 empezada en el instante 59 por el empleado 5
Tarea 10 empezada en el instante 74 por el empleado 4
Tarea 11 empezada en el instante 92 por el empleado 0
Tarea 12 empezada en el instante 97 por el empleado 2
Tarea 13 empezada en el instante 116 por el empleado 1
Tarea 14 empezada en el instante 134 por el empleado 1
Tarea 15 empezada en el instante 142 por el empleado 1
Tarea 16 empezada en el instante 161 por el empleado 4
Tarea 17 empezada en el instante 174 por el empleado 5
Tarea 18 empezada en el instante 17